# Water Quality and Storm Overflows

## Introduction

This Jupyter notebook will take you through creating an interactive map to display Waterbodies by Status (2015) and the Storm Overflow and Treated Discharge locations (NI Water).

## Datasets used

1. Surface Water Bodies Status 2015: [Available from DAERA OpenData](https://opendata-daerani.hub.arcgis.com/datasets/5229391e8cf142e384294ea12eace331_0/explore?location=54.633088%2C-6.837836%2C8.54) 
2. Modelled Spills May 2024 (Excel)* [Available from NI Water](https://www.niwater.com/siteFiles/resources/xls/NIWaterModelledSpillsMay2024v2.xlsx)
3. Wastewater Treatment Works Treated Discharges May 2024 (Excel)* [Available from NI Water](https://www.niwater.com/siteFiles/resources/xls/NIWaterTreatedDischargesMay2024.xlsx)
4. Event Duration Monitor Data (Excel)* [Available from NI Water](https://www.niwater.com/siteFiles/resources/2025/EventDurationMonitorDataApril2025.xlsx)

*More NI Water Information can be found on the [NI Water Storm Overflows page](https://www.niwater.com/storm/overflow/)

## Step One

You will need to import the packages panda, geopanda and folium. This can be done by running the cell below:

In [72]:
import pandas as pd
import geopandas as gpd
import folium

## Step Two

Add in the Surface Water Bodies (2015) Shapefile by running the below cell.
Note: If trying to replicate this you may need to first use the Repair Geometry tool in ArcGIS Pro, due to errors with "winding orders"

In [73]:
bodies = gpd.read_file('Datasets/Surface_Water_Bodies_2015_Status_Coloured.shp')

To confirm it loaded in correctly you can run the below cell to get a table view, which should show details for Mourne Coast and a couple of other Surface Waterbodies

In [ ]:
bodies.head()

## Step Three

Run the below code to generate the Surface Water Bodies map, with colour coding inspired by [NIEA Catchment Data Map Viewer](https://gis.daera-ni.gov.uk/arcgis/apps/webappviewer/index.html?id=16fddc459bd04d64b9e8f084f3a8e14a) symbology. E.g. Blue = High, and Red = Poor or Poor Ecological Potential (PEP).

In [ ]:
import folium

# Create a Folium map
m = folium.Map(location=[54.6, -5.9], zoom_start=10)

color_dict = {
    "BAD": "red", "BEP": "red",
    "POOR": "orange", "PEP": "orange",
    "MODERATE": "yellow", "MEP": "yellow", "MEP*": "yellow",
    "GOOD": "green" , "GEP": "green",
    "HIGH": "blue"
}
bodies["color"] = bodies["Stat_2015"].map(color_dict)

m = bodies.explore(color=bodies["color"])

m

## Step Four

Load in the Storm Overflows Discharge Points. Note: While NI Water mentions 2,440 Storm Overflows, some have multiple discharges meaning that 2,482 points should display on the Map.

You have two options here.

1. Run the cell below, to import data from a csv file, or
2. Clear the CSV script and copy the text below the cell for importing a shapefile of the data.


In [ ]:
df = pd.read_csv('Datasets/EGM722_ModelledSpillsFME.csv') # read the csv data

# create a new geodataframe
overflows = gpd.GeoDataFrame(df[['Name', 'CARID', '2023 Spill Count', '2024 Spill count', 'Monitored']], # use the csv data, but only the name/CARID/Spill Counts/Monitored columns
                            geometry=gpd.points_from_xy(df['X'], df['Y']), # set the geometry using points_from_xy where X and Y columns are the coordinates
                            crs='epsg:29902') # NI Water data uses the Irish Grid TM65 coordinate system

overflows = overflows.to_crs(epsg=4326) #converting it to WGS84 as it will help with later tasks.

overflows.head() # show that this has worked by displaying a table

# Note only 232 of the 2,440 Overflows (237 of 2,482 discharges) have monitors installed. So if you see NaN that's due to no data available (nulls)

## Step Five

To display the Overflows on the map run the cell below:

Note: Colours slightly different than those displayed on the [NI Water Storm Overflow Map](https://www.niwater.com/storm/overflow/)

In [ ]:
    # Add Storm Overflow points
for _, row in overflows.iterrows():
    folium.CircleMarker(
        location=[row["geometry"].y, row["geometry"].x],
        radius=1.5,
        color="black" if row["Monitored"] != "Yes" else "blue",
        fill=True,
        fill_opacity=0.8,
        popup=f"Name: {row['Name']}\n| Spills 2023: {row['2023 Spill Count']}\n| 2024 Spills: {row['2024 Spill count']}"
    ).add_to(m)
# Display the map
m

#Note you may notice that Ballyclose Street CSO displays in Ukraine - I suspect there is a translation error somewhere, because it's ~1,450miles away from Ballyclose Street Limavady.
#Could call it Ballyfar I guess? or given it's Star Wars weekend, at least the rest of the points didn't display in Alderaan places


## Step Six

To add Wastewater Treatment Works Points, to the data, run the cell below.

Note: NI Water data shows 1,022 WwTWs - but with some having multiple outlets (e.g. Belfast [3]) there are 1,042 points that will be added.

In [ ]:
df = pd.read_csv('Datasets/EGM722_TreatedDischargesFME.csv') # read the csv data

# create a new geodataframe
wwtws = gpd.GeoDataFrame(df[['Name', 'CARID']], # use the csv data, but only the Name/CARID columns
                            geometry=gpd.points_from_xy(df['X'], df['Y']), # set the geometry using points_from_xy where X and Y columns are the coordinates
                            crs='epsg:29902') # NI Water data uses the Irish Grid TM65 coordinate system

wwtws = wwtws.to_crs(epsg=4326) #converting it to WGS84 as it will help with later tasks.

wwtws.head() # show that this has worked by displaying a table

# Step 6

To add these points Treated Discharge points to the map run the cell below:

In [ ]:
# Add WwTW points
for _, row in wwtws.iterrows():
    folium.CircleMarker(
        location=[row["geometry"].y, row["geometry"].x],
        radius=1.5,
        color="purple",
        fill=True,
        fill_opacity=0.8,
        popup=row["Name"],
    ).add_to(m)
# Display the map
m

If the map symbology (colour and sizing) is to your liking you can save it to a HTML file by running the next cell

In [136]:
m.save('StatusMap.html')

# Footnote

# FME File names

You may notice the csv files having FME in the name. These were created by using FME to split the Duplicate discharge points from the tables provided by NI Water, and create standalone XY coordinate fields.

e.g. Ballygobackwards CSO 123456,234567 / 345678,456789

to:

Ballygobackwards CSO | X: 123456 | Y: 234567

Ballygobackwards CSO | X: 345678 | Y: 456789

# Inspiration

This could help outsiders to do some analysis with openly available data sources, to compare the likes of Spill Data with Water Quality. While it doesn't necessarily mean that NI Water Assets are the sole cause of the water quality results, there could be some contribution.